# 作业2-年收入判断

## 项目描述
二元分类是机器学习中最基础的问题之一，在这份教学中，你将学会如何实作一个线性二元分类器，来根据人们的个人资料，判断其年收入是否高于 50,000 美元。我们将以两种方法: logistic regression 与 generative model，来达成以上目的，你可以尝试了解、分析两者的设计理念及差别。
实现二分类任务：
* 个人收入是否超过50000元？

## 数据集介绍
这个资料集是由UCI Machine Learning Repository 的Census-Income (KDD) Data Set 经过一些处理而得来。为了方便训练，我们移除了一些不必要的资讯，并且稍微平衡了正负两种标记的比例。事实上在训练过程中，只有 X_train、Y_train 和 X_test 这三个经过处理的档案会被使用到，train.csv 和 test.csv 这两个原始资料档则可以提供你一些额外的资讯。
* 已经去除不必要的属性。
* 已经平衡正标和负标数据之间的比例。

**特征格式**
1. train.csv，test_no_label.csv。
* 基于文本的原始数据
* 去掉不必要的属性，平衡正负比例。
2. X_train, Y_train, X_test(测试)
* train.csv中的离散特征=>在X_train中onehot编码(学历、状态...)
* train.csv中的连续特征 => 在X_train中保持不变(年龄、资本损失...)。
* X_train, X_test : 每一行包含一个510-dim的特征，代表一个样本。
* Y_train: label = 0 表示 "<=50K" 、 label = 1 表示 " >50K " 。

## 项目要求
1. 请动手编写 gradient descent 实现 logistic regression
1. 请动手实现概率生成模型。
1. 单个代码块运行时长应低于五分钟。
1. 禁止使用任何开源的代码(例如，你在GitHub上找到的决策树的实现)。

## 数据准备
项目数据保存在：work/data/ 目录下。

## 环境配置/安装
无

In [ ]:
# 下面该你动手啦！
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import  metrics

In [ ]:
X = pd.read_csv('work/data/X_train')
X_test = pd.read_csv('work/data/X_test')
Y_test = pd.read_csv('work/output_logistic.csv')
Y_test = Y_test.iloc[:,1:].to_numpy()
X_test = X_test.iloc[:,1:].to_numpy()
X = X.iloc[:,1:].to_numpy()
Y = pd.read_csv('work/data/Y_train')
Y = Y.iloc[:,1:].to_numpy()
MinMax = preprocessing.MinMaxScaler()
x_scale = MinMax.fit_transform(X)
data_x = x_scale
data_y = Y
test_x = MinMax.fit_transform(X_test)

In [ ]:
### for SGD
def sigmoid(x):
    y = 1/(1+np.exp(x))
    return y


def mini_batch(batch_size, data_X, data_Y):
    idx = np.arange(0, data_X.shape[0])
    np.random.shuffle(idx)
    idx = idx[0:batch_size]

    return data_X[idx,:], data_Y[idx,:]

class logistics_regression():

    def __init__(self,X_train, label):
        self.X = X_train
        self.Y = label
        self.weight1 = np.random.normal(0,0.5,[1,self.X.shape[1]])
        self.b = np.random.normal(0,0.5,[1,1])
        self.lr = 0.1
        self.g_w = np.zeros(self.weight1.shape)
        self.g_b = np.zeros(self.b.shape)
        self.loss = 0
        self.eps = 0.00001

    def train(self):
        m = self.X.shape[0]
        # loss = 0


        for i in range(m):
            for j in range(5):
                output = self.forward(self.X[i,:])
                gw = (output - self.Y[i,:])*self.X[i,:]
                self.g_w += gw**2
                self.weight1 = self.weight1 + self.lr * gw/(np.sqrt(self.g_w)+self.eps)
                # self.weight1 = self.weight1 + self.lr * gw
                gb = (output - self.Y[i,:])
                self.g_b += gb**2
                self.b = self.b + self.lr * gb/(np.sqrt(self.g_b)+self.eps)
                # self.b = self.b + self.lr * gb

        for j in range(m):
            self.loss += self.Y[j,:]*np.log(self.forward(self.X[j,:]) + self.eps) + (1 - self.Y[j,:])*np.log(1 - self.forward(self.X[j,:]) + self.eps)   
        self.loss = -self.loss

    def forward(self, x):
        y_pred = sigmoid(np.dot(self.weight1, x) + self.b)
        # y_pred = np.dot(self.weight1, x) + np.dot(self.weight2, x**2) + self.b
        return y_pred 



In [11]:
iter = 50
batch_size = 10000

# x = np.array([[0,0,0],[1,1,1]])
# y = np.array([[0],[1]])
# data_x = x
# data_y = y

x_batch, y_batch = mini_batch(batch_size, data_x,data_y)
LG = logistics_regression(x_batch,y_batch)
for epoch in range(iter):
    x_batch, y_batch = mini_batch(batch_size, data_x,data_y)
    LG.X = x_batch
    LG.Y = y_batch
    for i in range(10):
        LG.loss = 0
        LG.train()
        if i % 100 == 0:
            print(LG.loss)

[[2809.15164883]]


In [ ]:
x_batch, y_batch = mini_batch(1, data_x,data_y)
LG = logistics_regression(x_batch,y_batch)
LG.weight1 = np.load('weight.npy')
LG.b = np.load('bias.npy')
op = 1
LG.forward(data_x[op,:]), data_y[op,:]

(array([[0.2564197]]), array([0]))

In [ ]:
# LG = logistics_regression(x_batch,y_batch)
op = 27619
LG.forward(test_x[op,:]), Y_test[op,:]

(array([[0.84450424]]), array([1]))

In [ ]:
scores = LG.forward(test_x.T)
result = scores.copy()
result[np.where(scores >= 0.5)] = 1
result[np.where(scores < 0.5)] = 0

In [ ]:
print('acc:', 1 - np.sum((result - Y_test.T)**2)/Y_test.shape[0])
print('auc:', metrics.roc_auc_score(Y_test, scores.T))

acc: 0.9501122293823764
auc: 0.9832468647415978


In [ ]:
csv_result = pd.DataFrame(result.T)
csv_result.columns = ["label"]
csv_result["label"] = csv_result["label"].astype(int)
# csv_result.index = ['id' + str(i) for i in range(result.shape[1])]
print(csv_result)
csv_result.to_csv("submission.csv")

       label
0          0
1          1
2          0
3          0
4          0
...      ...
27617      0
27618      0
27619      1
27620      0
27621      0

[27622 rows x 1 columns]


In [ ]:
np.save("weight.npy", LG.weight1)
np.save("bias.npy", LG.b)